In [1]:
import dexpy.factorial
import dexpy.power
import pandas as pd
import numpy as np

In [2]:
factors = {
    "lysate_aspirate_height_inc" : 0.4,
    "lysate_aspirate_rate" : 0.2,
    "lysate_dispense_rate" : 0.1,
    
    "substrates_aspirate_height_inc" : 0.7,
    "substrates_aspirate_rate" : 1,
    "substrates_dispense_rate" : 1
    }

# Factor Metadata
Simply gets some metadata about the factors

In [3]:
# number of factors as an integer
number_of_factors = len(factors)

# a python list of the names of the factors by getting the dictionary keys
names_of_factors = list(factors.keys())

# Choosing and generating an experimental design dynamically

Placeholder: full factorial

### Full factorial run generation formula:

Number of runs = 2** number of factors

![Doe_resolution_image](img/doe_resolution_table.png)

In [4]:

# initialise experimental design
experimental_design = dexpy.factorial.build_factorial(number_of_factors, 2**number_of_factors)

# label columns with factor names
experimental_design.columns = names_of_factors

In [5]:
experimental_design

,lysate_aspirate_height_inc,lysate_aspirate_rate,lysate_dispense_rate,substrates_aspirate_height_inc,substrates_aspirate_rate,substrates_dispense_rate
0,-1,-1,-1,-1,-1,-1
1,-1,-1,-1,-1,-1,1
2,-1,-1,-1,-1,1,-1
3,-1,-1,-1,-1,1,1
4,-1,-1,-1,1,-1,-1
...,...,...,...,...,...,...
59,1,1,1,-1,1,1
60,1,1,1,1,-1,-1
61,1,1,1,1,-1,1
62,1,1,1,1,1,-1


# Aliasing

We want to remove rows that are duplicated and columns with are repeated to ensure our experiment is efficent as possible

In [6]:
#rows
# creates a bool series of True/False row-is-duplicate and then filters the DF by dropping those rows
experimental_design = experimental_design[~experimental_design.duplicated()]

#columns
# Transpose the df to allow the same simple function to be used row-wise and then transpose back
experimental_design = experimental_design.T
experimental_design = experimental_design[~experimental_design.duplicated()]
experimental_design = experimental_design.T
experimental_design

,lysate_aspirate_height_inc,lysate_aspirate_rate,lysate_dispense_rate,substrates_aspirate_height_inc,substrates_aspirate_rate,substrates_dispense_rate
0,-1,-1,-1,-1,-1,-1
1,-1,-1,-1,-1,-1,1
2,-1,-1,-1,-1,1,-1
3,-1,-1,-1,-1,1,1
4,-1,-1,-1,1,-1,-1
...,...,...,...,...,...,...
59,1,1,1,-1,1,1
60,1,1,1,1,-1,-1
61,1,1,1,1,-1,1
62,1,1,1,1,1,-1
